#### Installation des modules nécessaires au redimensionnement

Code adapté à partir du script de création de *dataset* proposé par l'équipe de Silknow et disponible sur leur [GitHub]( https://github.com/silknow/image-classification/blob/master/silknow_image_classification/src/DatasetCreation.py).

In [2]:
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


#### Vérification du chemin d'accès et des images

In [3]:
pwd

'/Users/lise/memoire/scripts'

In [8]:
cd '/Users/lise/memoire/'

/Users/lise/memoire


#### Redimensionnement

In [11]:
def rescaleImages(directory):
    imgpath_load = directory + r"/img_unscaled/"
    imgpath_save = directory + r"/img_rescaled/"
    if not os.path.exists(imgpath_save):
        os.makedirs(imgpath_save)

    imglist = os.listdir(imgpath_load)
    print("\nTotal number of images: ", len(imglist))

    # for img_file in tqdm(imglist):
    deadlist_load = []
    deadlist_scale = []
    print("\nIterating over images (download): ")
    for img_file in tqdm(imglist, total=len(imglist)):

        # Skip images that are already scaled
        if os.path.exists(imgpath_save + img_file): continue

        # Try to open images, skip else
        try:
            img = plt.imread(imgpath_load + img_file)
        except:
            deadlist_load += [img_file]
            continue

        try:
            # get dimensions of image
            if len(img.shape) == 2:
                width, heigth = img.shape
                img_new = np.zeros([img.shape[0], img.shape[1], 3], dtype=np.uint8)
                img_new[:, :, 0], img_new[:, :, 1], img_new[:, :, 2] = img, img, img
                img = img_new
            elif len(img.shape) == 3:
                width, heigth, _ = img.shape

            smaller_side = np.minimum(heigth, width)
            scale_factor = 448. / smaller_side

                # If Downscaling, apply gaussian blur
            if scale_factor < 1.:
                sigma = 1. / scale_factor
                kernelsize = int(sigma * 6) + (1 - (int(sigma * 6) % 2))
                img = cv2.GaussianBlur(img, (kernelsize, kernelsize), sigma)

            img_new = cv2.resize(img, (int(heigth * scale_factor), int(width * scale_factor)),
                                     interpolation=cv2.INTER_CUBIC)
            plt.imsave(imgpath_save + img_file, img_new)
        except:
            print("Erreur de mise à l'échelle:"+img_file)
            deadlist_scale += [img_file]

    if len(deadlist_load) > 0:
        print("\nThe following images could not be opened: ")
        for deadload in deadlist_load:
            print(deadload)
    if len(deadlist_scale) > 0:
        print("\nThe following images could not be scaled: ")
        for _ in deadlist_scale:
            print(deadlist_scale)

In [12]:
rescaleImages("images")


Total number of images:  273

Iterating over images (download): 


100%|██████████| 273/273 [00:23<00:00, 11.70it/s]


In [3]:
def noms_images_txt(repertoire, nom_document):
    contenu = ""

    if not os.path.isdir(repertoire):
        print("Le répertoire spécifié n'existe pas.")
        return

    fichiers = sorted(os.listdir(repertoire))
    for fichier in fichiers:
        chemin_fichier = os.path.join(repertoire, fichier)
        if os.path.isfile(chemin_fichier):
            contenu += f"{fichier}\n"

    try:
        with open(nom_document, "w") as fichier_txt:
            fichier_txt.write(contenu)
        print(f"Le document '{nom_document}' a été créé avec succès.")
    except IOError:
        print("Une erreur s'est produite lors de l'écriture du document.")

In [5]:
noms_images_txt("../images/img_rescaled/", "noms_images.txt")

Le document 'noms_images.txt' a été créé avec succès.
